# Imports necessários

In [1]:
import pandas as pd
import numpy as np
import funcoes_importantes as fi

In [2]:
fp = fi.funcoes_main()

In [13]:
a = pd.DataFrame([mcov_1.values,mcov_2.values,mcov_3.values,mcov_4.values,mcov_5.values,mcov_6.values])
a.iloc[0].values

array([array([[0.36864032319151513, 0.10036611538947127, 0.09632622143668756,
        ..., -0.010857214998107599, -0.021461936624163833,
        0.8756470142658462],
       [0.10036611538947127, 0.37646761772503495, 0.14833985607877784,
        ..., 0.002777427092538783, -0.030046711273828672,
        -0.36510541598286206],
       [0.09632622143668756, 0.14833985607877784, 0.3289988637798256,
        ..., 0.04380760005049938, -0.036737785633126956,
        0.0022724403484522208],
       ...,
       [-0.010857214998107599, 0.002777427092538783, 0.04380760005049938,
        ..., 0.4817573538694586, -0.029920464587804747,
        0.9915414720363678],
       [-0.021461936624163833, -0.030046711273828672,
        -0.036737785633126956, ..., -0.029920464587804747,
        0.055043555106678715, -0.9176871607120769],
       [0.8756470142658462, -0.36510541598286206, 0.0022724403484522208,
        ..., 0.9915414720363678, -0.9176871607120769, 250.89332155031175]],
      dtype=object)], dtype=ob

In [16]:
mcov_1_lambda = fp.correcao_lambda(a.iloc[0].values)
mcov_1_lambda

array([[array([[0.37864032319151514, 0.11036611538947126, 0.10632622143668756,
        ..., -0.0008572149981075985, -0.011461936624163832,
        0.8856470142658462],
       [0.11036611538947126, 0.38646761772503496, 0.15833985607877785,
        ..., 0.012777427092538783, -0.02004671127382867,
        -0.35510541598286205],
       [0.10632622143668756, 0.15833985607877785, 0.33899886377982563,
        ..., 0.05380760005049938, -0.026737785633126954,
        0.012272440348452221],
       ...,
       [-0.0008572149981075985, 0.012777427092538783,
        0.05380760005049938, ..., 0.4917573538694586,
        -0.019920464587804748, 1.0015414720363678],
       [-0.011461936624163832, -0.02004671127382867,
        -0.026737785633126954, ..., -0.019920464587804748,
        0.06504355510667871, -0.9076871607120769],
       [0.8856470142658462, -0.35510541598286205, 0.012272440348452221,
        ..., 1.0015414720363678, -0.9076871607120769, 250.90332155031174]],
      dtype=object)]], dtype=ob

In [3]:
# Antes de tudos normalizar z-score os dados com as info do dados de treino(mean e std)
# iSSO TUDO PARA 6 CLASSES

def qda(X,y):
    # 1- calcula a probilidade a priori[1x1]
    probs_p = fp.prob_a_priori(X)

    # 2- vetor médio por classe[1x34]
    vt_medio = fp.vetor_medio_p_classe(X)

    
    # 3- matriz covariancia por classe[34x34]
    mcov_1 = fp.matriz_covariancia(X.query('c35==1'))
    mcov_2 = fp.matriz_covariancia(X.query('c35==2'))
    mcov_3 = fp.matriz_covariancia(X.query('c35==3'))
    mcov_4 = fp.matriz_covariancia(X.query('c35==4'))
    mcov_5 = fp.matriz_covariancia(X.query('c35==5'))
    mcov_6 = fp.matriz_covariancia(X.query('c35==6'))
    a = pd.DataFrame([mcov_1.values,mcov_2.values,mcov_3.values,mcov_4.values,mcov_5.values,mcov_6.values])
    
    # 4- Fazer a correção lamdba A = A + xI na matriz de covariancia
    mcov_1_lambda = fp.correcao_lambda(mcov_1)
    mcov_2_lambda = fp.correcao_lambda(mcov_2)
    mcov_3_lambda = fp.correcao_lambda(mcov_3)
    mcov_4_lambda = fp.correcao_lambda(mcov_4)
    mcov_5_lambda = fp.correcao_lambda(mcov_5)
    mcov_6_lambda = fp.correcao_lambda(mcov_6)

    # 5- Calcular a inversa da matriz de covairância regulada lambda
    mcov_1_lambda_inv = fp.matriz_inversa(mcov_1_lambda)
    mcov_2_lambda_inv = fp.matriz_inversa(mcov_2_lambda)
    mcov_3_lambda_inv = fp.matriz_inversa(mcov_3_lambda)
    mcov_4_lambda_inv = fp.matriz_inversa(mcov_4_lambda)
    mcov_5_lambda_inv = fp.matriz_inversa(mcov_5_lambda)
    mcov_6_lambda_inv = fp.matriz_inversa(mcov_6_lambda)

    # 6- Calcular o determinante de cada matriz de covariância regulada
    det_1 = fp.determinante_matriz_covariancia(mcov_1_lambda)
    det_2 = fp.determinante_matriz_covariancia(mcov_2_lambda)
    det_3 = fp.determinante_matriz_covariancia(mcov_3_lambda)
    det_4 = fp.determinante_matriz_covariancia(mcov_4_lambda)
    det_5 = fp.determinante_matriz_covariancia(mcov_5_lambda)
    det_6 = fp.determinante_matriz_covariancia(mcov_6_lambda)


    # Teste, pegar agora um elementro da base de teste e aplicar
    # min: ln|det(mcov_classe_i)| + 
    #       (elem_teste-vetor_medio_classe_i).T*mcov_classe_i_inv*(elem_teste-vetor_md_classe_i)
    #       - 2*ln(P_priori(classe_i)) 

    # elem_teste é X da questão, literalmente
    # o que o min retorna?
    # min retorna um valor 1,2,3,4,5,6 sendo este a classe

    aux = []
    for i in y.index.values:
        valores = []
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio.iloc[0],mcov_1_lambda_inv,det_1,probs_p[0]))    
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio.iloc[1],mcov_2_lambda_inv,det_2,probs_p[1]))
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio.iloc[2],mcov_3_lambda_inv,det_3,probs_p[2]))    
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio.iloc[3],mcov_4_lambda_inv,det_4,probs_p[3]))
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio.iloc[4],mcov_5_lambda_inv,det_5,probs_p[4]))
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio.iloc[5],mcov_6_lambda_inv,det_6,probs_p[5]))
        aux.append(valores.index(min(valores))+1)
    print('Acrurácia de ',fp.get_acc(aux,y['c35'])*100,'%')
    return fp.get_acc(aux,y['c35'])

In [4]:
X,y = fp.kfold_shuffle(5)
qda(X,y)

Acrurácia de  97.22 %


0.9722

In [5]:
# qda_accs = []
# for _ in range(50):
#     X,y = fp.kfold_shuffle(5)
#     qda_accs.append(qda(X,y))
# print('Acurácia média de ',np.mean(qda_accs)*100,'%')